In [27]:
from PIL import Image
from PIL import ImageOps 
import numpy as np 
import os 

import glob
import re

In [28]:
from imageToPixels import *

In [29]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)

In [4]:
# def load_image2(infilename) :
#     '''  
#     convert image file to pixels
#     load_image2('fileName')
#     '''
#     img = Image.open(infilename).convert('L')
#     data = np.array(img)
#     return data


# def ageAtPhoto(fileName):
#     '''
#     get age at time of photo 
#     ageAtPhoto('full_path_to_file')
#     10049200_1891-09-16_1958.jpg
#     yob is 1891
#     dtpt is 1958 
#     '''
#     tailPath = os.path.basename(fileName)
#     matchObj = re.search(r'^[^_]+_([^_]+)_[^_]+$', tailPath)
#     yob = int(matchObj.group(1)[:4])
#     nojpg = matchObj.group(0)[:-4]
#     dtpt = int(nojpg[-4:])
#     currAge = abs(dtpt - yob)
#     return currAge


# def convertToNumpy(folder):
#     '''
#     get pixels and age for each image in a folder
#     x_values, y_values = convertToNumpy(fileNames)    
#     '''
#     pixels = []
#     ages = []
#     for fileName in folder:
#         if fileName.endswith(".jpg" or ".jpeg" or ".png"):
#             pixels.append(np.ravel(load_image2(fileName)))
#             ages.append(ageAtPhoto(fileName))
#     return pixels, ages


In [30]:
folderName = '/Users/davidkes/classes/697/Project/wiki_crop/' 
fileNames = glob.glob(folderName +'*_resized/*.jpg')
# only test 20 files for now
NumberOfFileToBetested =300

x_values, y_values = convertToNumpy(fileNames[:NumberOfFileToBetested])

# print x_values
# print y_values

In [32]:
flat_rdd = sc.parallelize(x_values).map(lambda x : x.tolist()).map(lambda x: [int(element) for element in x])
# len(flat_rdd.take(5)[0])
age_rdd = sc.parallelize(y_values).map(lambda x:int(x))
# age_rdd.take(5)
combined = flat_rdd.zip(age_rdd)
# combined.collect()[0][1]

16384

In [35]:
#Create a DataFrame
from pyspark.sql.types import *
from pyspark.sql import Row
penschema = StructType([
   StructField("features", ArrayType(elementType=IntegerType(),containsNull=False),True),
   StructField("label", IntegerType(),True)
])
dfpen1 = sqlContext.createDataFrame(combined.map(lambda x : Row(x[0][:],x[1])), penschema)
dfpen1.show()

In [37]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
dfpen1 = dfpen1.select(list_to_vector_udf(dfpen1["features"]).alias("features"),'label')

pendtsets = dfpen1.randomSplit([0.8, 0.2])
pendttrain = pendtsets[0].cache()
pendtvalid = pendtsets[1].cache()

In [38]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(regParam=0.01, maxIter=10, fitIntercept=True)
lrmodel = lr.fit(pendttrain)

In [39]:
validpredict = lrmodel.transform(pendtvalid)
validpredict.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[12.0,12.0,12.0,1...|   35|[7.38833377810578...|[0.00148664721115...|      51.0|
|[25.0,25.0,25.0,2...|   26|[4.20361383406276...|[1.14284124341466...|      30.0|
|[61.0,61.0,61.0,6...|   24|[4.09090963001608...|[1.13888916099408...|      21.0|
|[62.0,62.0,62.0,6...|   66|[2.71120730668195...|[9.94297652392477...|      46.0|
|[143.0,143.0,143....|   60|[13.2968108020684...|[0.00751876820522...|      31.0|
|[165.0,165.0,165....|   40|[8.60832564226181...|[1.65832346658064...|      21.0|
|[171.0,170.0,169....|   23|[11.9945858863914...|[0.01281040676709...|      27.0|
|[176.0,176.0,176....|   26|[11.5420644153596...|[8.73401397146175...|      20.0|
|[213.0,213.0,213....|   24|[8.12484575481163...|[0.00119119511604...|      20.0|
|[215.0,221.0,21

In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
bceval = MulticlassClassificationEvaluator()
print ("F1 = %.4f" % bceval.evaluate(validpredict))

F1 = 0.0242
